# Fine-tuning a model with the Trainer API

Hugging Face Transformers provides a Trainer class to help you fine-tune any of the pretrained models it provides on your dataset. Once you’ve done all the data preprocessing work in the last section, you have just a few steps left to define the `Trainer`. The hardest part is likely to be preparing the environment to run `Trainer.train()`.

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [2]:
sample = tokenized_datasets["train"][0]
print("The model will input the sentence1 and sentence2 strings tokenized together")
print(f"Sentence1: {sample['sentence1']}")
print(f"Sentence2: {sample['sentence2']}")
print(f"And expects in the batch a 'label'. In this case: {sample['label']}")

The model will input the sentence1 and sentence2 strings tokenized together
Sentence1: Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .
Sentence2: Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .
And expects in the batch a 'label'. In this case: 1


# Training

The first step before we can define our `Trainer` is to define a `TrainingArguments` class that will contain all the hyperparameters the Trainer will use for training and evaluation. The only argument you have to provide is a directory where the trained model will be saved, as well as the checkpoints along the way. For all the rest, you can leave the defaults, which should work pretty well for a basic fine-tuning.

In [3]:
from transformers import TrainingArguments

training_args = TrainingArguments("tmp")

The second step is to define our model. We will use the `AutoModelForSequenceClassification` class, with two labels:

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

You get a warning after instantiating this pretrained model. This is because BERT has not been pretrained on classifying pairs of sentences, so **the head of the pretrained model has been discarded and a new head suitable for sequence classification has been added** instead. The warnings indicate that some weights were not used (the ones corresponding to the dropped pretraining head) and that some others were randomly initialized (the ones for the new head). It concludes by encouraging you to train the model, which is exactly what we are going to do now.

Once we have our model, we can define a `Trainer` by passing it all the objects constructed up to now — the model, the `training_args`, the training and validation `datasets`, our `data_collator`, and our `tokenizer`:

In [5]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

To fine-tune the model on our dataset, we just have to call the `train()` method of our `Trainer`:

In [6]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: marioparreno. Use `wandb login --relogin` to force relogin


  0%|          | 0/1377 [00:00<?, ?it/s]

{'loss': 0.5343, 'grad_norm': 1.404845952987671, 'learning_rate': 3.184458968772695e-05, 'epoch': 1.09}
{'loss': 0.3391, 'grad_norm': 57.90313720703125, 'learning_rate': 1.3689179375453886e-05, 'epoch': 2.18}
{'train_runtime': 97.3555, 'train_samples_per_second': 113.029, 'train_steps_per_second': 14.144, 'train_loss': 0.3704173047211868, 'epoch': 3.0}


TrainOutput(global_step=1377, training_loss=0.3704173047211868, metrics={'train_runtime': 97.3555, 'train_samples_per_second': 113.029, 'train_steps_per_second': 14.144, 'total_flos': 405114969714960.0, 'train_loss': 0.3704173047211868, 'epoch': 3.0})

This will start the fine-tuning (which should take a couple of minutes on a GPU) and report the training loss every 500 steps. It won’t, however, tell you how well (or badly) your model is performing. This is because:

1. We didn’t tell the Trainer to evaluate during training by setting `evaluation_strategy` to either "steps" (evaluate every `eval_steps`) or "`epoch`" (evaluate at the end of each epoch).
2. We didn’t provide the `Trainer` with a `compute_metrics()` function to calculate a metric during said evaluation (otherwise the evaluation would just have printed the loss, which is not a very intuitive number).

# Evaluation

Let’s see how we can build a useful `compute_metrics()` function and use it the next time we train. The function must take an `EvalPrediction` object (which is a named tuple with a `predictions` field and a `label_ids` field) and will **return a dictionary mapping strings to floats** (the strings being the names of the metrics returned, and the floats their values). To get some predictions from our model, we can use the `Trainer.predict()` command:

In [7]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)
print(f"Predictions:\n{predictions.predictions[:5]}")
print(f"Labels:\n{predictions.label_ids[:5]}")

  0%|          | 0/51 [00:00<?, ?it/s]

(408, 2) (408,)
Predictions:
[[-3.0765405  2.3506184]
 [ 2.414047  -2.623411 ]
 [ 1.3966411 -1.9060795]
 [-3.0748508  2.3500826]
 [ 2.1715033 -2.4174147]]
Labels:
[1 0 0 1 0]


The output of the `predict()` method is another named tuple with **three fields**: `predictions`, `label_ids`, and `metrics`. The metrics field will just contain the loss on the dataset passed, as well as some time metrics (how long it took to predict, in total and on average). Once we complete our `compute_metrics()` function and pass it to the `Trainer`, that field will also contain the metrics returned by `compute_metrics()`.

As you can see, `predictions` is a two-dimensional array with shape 408 x 2 (408 being the number of elements in the dataset we used). Those are the **logits** for each element of the dataset we passed to `predict()`. To transform them into predictions that we can compare to our labels, we need to take the index with the maximum value on the second axis:

We can now compare those preds to the labels. To build our `compute_metric()` function, we will rely on the metrics from the Hugging Face Evaluate library. We can load the metrics associated with the MRPC dataset as easily as we loaded the dataset, this time with the `evaluate.load()` function. The object returned has a `compute()` method we can use to do the metric calculation:

In [8]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(
    predictions=predictions.predictions.argmax(axis=1),
    references=predictions.label_ids
)

{'accuracy': 0.8382352941176471, 'f1': 0.8877551020408163}

Those are the two metrics used to evaluate results on the MRPC dataset for the GLUE benchmark.

Wrapping everything together, we get our `compute_metrics()` function:

In [9]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = logits.argmax(axis=1)
    return metric.compute(predictions=predictions, references=labels)

And to see it used in action to report metrics at the end of each epoch, here is how we define a new `Trainer` with this `compute_metrics()` function

In [ ]:
training_args = TrainingArguments("tmp", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Note that we create a new `TrainingArguments` with its `evaluation_strategy` set to "`epoch`" and a new model — otherwise, we would just be continuing the training of the model we have already trained. To launch a new training run, we execute:

In [11]:
trainer.train()

  0%|          | 0/1377 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]

{'eval_loss': 0.4018109142780304, 'eval_accuracy': 0.8529411764705882, 'eval_f1': 0.8983050847457628, 'eval_runtime': 1.4484, 'eval_samples_per_second': 281.692, 'eval_steps_per_second': 35.212, 'epoch': 1.0}
{'loss': 0.525, 'grad_norm': 73.473876953125, 'learning_rate': 3.184458968772695e-05, 'epoch': 1.09}


  0%|          | 0/51 [00:00<?, ?it/s]

{'eval_loss': 0.4361107051372528, 'eval_accuracy': 0.8823529411764706, 'eval_f1': 0.9172413793103448, 'eval_runtime': 1.4398, 'eval_samples_per_second': 283.376, 'eval_steps_per_second': 35.422, 'epoch': 2.0}
{'loss': 0.3017, 'grad_norm': 0.7111353278160095, 'learning_rate': 1.3689179375453886e-05, 'epoch': 2.18}


  0%|          | 0/51 [00:00<?, ?it/s]

{'eval_loss': 0.5136808753013611, 'eval_accuracy': 0.8848039215686274, 'eval_f1': 0.9185441941074524, 'eval_runtime': 1.4341, 'eval_samples_per_second': 284.498, 'eval_steps_per_second': 35.562, 'epoch': 3.0}
{'train_runtime': 95.9325, 'train_samples_per_second': 114.706, 'train_steps_per_second': 14.354, 'train_loss': 0.3546374950533599, 'epoch': 3.0}


TrainOutput(global_step=1377, training_loss=0.3546374950533599, metrics={'train_runtime': 95.9325, 'train_samples_per_second': 114.706, 'train_steps_per_second': 14.354, 'total_flos': 405114969714960.0, 'train_loss': 0.3546374950533599, 'epoch': 3.0})